In [1]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import random
import copy
import ipython_bell

from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM
from astropy.visualization import make_lupton_rgb

import lenstronomy.Util.param_util as param
import lenstronomy.Util.util as util
from lenstronomy.SimulationAPI.sim_api import SimAPI
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
DES_camera = {'read_noise': 7.,  # std of noise generated by read-out (in units of electrons)
               'pixel_scale': 0.2637,  # scale (in arcseconds) of pixels
               'ccd_gain': 6.  # electrons/ADU (analog-to-digital unit). A gain of 8 means that the camera digitizes the CCD signal so that each ADU corresponds to 8 photoelectrons.
              }

DES_g_band_obs = {'exposure_time': 90.,  # exposure time per image (in seconds)
                   'sky_brightness': 35.01,  # sky brightness (in magnitude per square arcseconds)
                   'magnitude_zero_point': 30,  # magnitude in which 1 count per second per arcsecond square is registered (in ADU's)
                   'num_exposures': 7,  # number of exposures that are combined
                   'seeing': 1.12,  # full width at half maximum of the PSF (if not specific psf_model is specified)
                   'psf_type': 'GAUSSIAN',  # string, type of PSF ('GAUSSIAN' and 'PIXEL' supported)
                   'kernel_point_source': None  # 2d numpy array, model of PSF centered with odd number of pixels per axis (optional when psf_type='PIXEL' is chosen)
                  }

DES_r_band_obs = {'exposure_time': 90.,  
                   'sky_brightness': 34.7,  
                   'magnitude_zero_point': 30,  
                   'num_exposures': 7,  
                   'seeing': 1.12,  
                   'psf_type': 'GAUSSIAN', 
                  }

DES_i_band_obs = {'exposure_time': 90.,  
                   'sky_brightness': 35.1,  
                   'magnitude_zero_point': 30,  
                   'num_exposures': 7,  
                   'seeing': 1.12,  
                   'psf_type': 'GAUSSIAN', 
                  }

numpix = 46 # 

kwargs_g_band = util.merge_dicts(DES_camera, DES_g_band_obs)
kwargs_r_band = util.merge_dicts(DES_camera, DES_r_band_obs)
kwargs_i_band = util.merge_dicts(DES_camera, DES_i_band_obs)

kwargs_numerics = {'point_source_supersampling_factor': 10}

cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.)


In [3]:
# Function to pick a position on a caustic
def pick_center(ra_caustic_list, dec_caustic_list):
    l = random.randint(0, len(ra_caustic_list)-1)
    return(ra_caustic_list[l], dec_caustic_list[l])

In [4]:
#Writes 3 files for each simulation (one for each band). Folder is the ID, name n_g.fits
def write_files(ID, image_g, image_r, image_i, parameters):
    path = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Sim/'
    folder = str(ID)
    if not os.path.exists(path + folder):
        os.makedirs(path + folder)
        n = 0
    else:
        n = len(os.listdir(path + folder))/3 #/3
    
    parameters = Table(data=parameters, dtype=[np.float64]*5)
        
    primary = fits.PrimaryHDU(image_g)
    table = fits.BinTableHDU(data = parameters)
    hdul = fits.HDUList([primary, table])
    hdul.writeto(path + folder + '/' + str(int(n+1)) + '_g.fits')
    
    primary = fits.PrimaryHDU(image_r)
    hdul = fits.HDUList([primary])
    hdul.writeto(path + folder + '/' + str(int(n+1)) + '_r.fits')
    
    primary = fits.PrimaryHDU(image_i)
    hdul = fits.HDUList([primary])
    hdul.writeto(path + folder + '/' + str(int(n+1)) + '_i.fits')
    

In [5]:
#Calculating ~total flux
def calculate_flux(object, band):
    flux = np.sum(object[band])
    return(flux)

#Calculating AB magnitude
def calculate_magnitude(object, band):
    f = calculate_flux(object, band)
    m = -2.5*np.log10(f*10**(-12))
    return(m)

# For source distributions: from redshift get magnitude, sersic radius & ellipticity
path = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/'
filename = 'source_distributions.csv'
data_dist = pd.read_csv(path + filename)
data_dist = data_dist[data_dist['MAG_PSF_G'] < 30.] [data_dist['MAG_PSF_R'] < 30.] [data_dist['MAG_PSF_I'] < 30.]
data_dist = data_dist[data_dist['SOF_CM_G_1'] > -100.][data_dist['SOF_CM_G_2'] > -100.]
data_dist = data_dist[data_dist['DNF_ZMEAN_SOF'] > 0.01][data_dist['DNF_ZMEAN_SOF'] < 2.9]
data_dist = data_dist.sort_values('DNF_ZMEAN_SOF').reset_index()

dz, dm = 0.1, 0.1 # Range of redshift and magnitude for filtering

#returns magnitude in g, r, i bands and r sersic
def distribution(z, mmax = 25.5, data=data_dist):
    new_data = data[data['DNF_ZMEAN_SOF'] > z - dz] [data['DNF_ZMEAN_SOF'] < z + dz] 
    m = new_data.sample()['MAG_PSF_G'].values[0]
    while(m > mmax):
        m = new_data.sample()['MAG_PSF_G'].values[0]
    new_data = new_data[new_data['MAG_PSF_G'] > m - dm] [new_data['MAG_PSF_G'] < m + dm]
    random_object = new_data.sample()
    mg, mr, mi = random_object['MAG_PSF_G'].values[0], random_object['MAG_PSF_R'].values[0], random_object['MAG_PSF_I'].values[0]
    rg, rr, ri = random_object['FLUX_RADIUS_G'].values[0], random_object['FLUX_RADIUS_R'].values[0], random_object['FLUX_RADIUS_I'].values[0]
    e1, e2 = random_object['SOF_CM_G_1'].values[0], random_object['SOF_CM_G_2'].values[0]
    magnitude = {'mg': mg, 'mr': mr, 'mi': mi}
    radius = {'rg': rg*DES_camera['pixel_scale'], 'rr': rr*DES_camera['pixel_scale'], 'ri': ri*DES_camera['pixel_scale']}
    ellipticity = {'e1': e1, 'e2': e2}
    return(magnitude, radius, ellipticity)

#z = 1.7 Max


In [6]:
def simulation(model, coadd_id, redshifts, lens, cuts, double):
   
    kwargs_model_physical = {'lens_model_list': ['SIE'],  # list of lens models to be used
                          'lens_redshift_list': [redshifts['lens']],  # list of redshift of the deflections
                          # list of extended source models to be used
                          'source_light_model_list': ['SERSIC_ELLIPSE'],  
                          # list of redshfits of the sources in same order as source_light_model_list
                          'source_redshift_list': [redshifts['source1']],  
                          'cosmo': cosmo,  # astropy.cosmology instance
                          # redshift of the default source (if not further specified by 'source_redshift_list')
                          'z_source': redshifts['source1']} 
                           #and also serves as the redshift of lensed point sources}

    if(double == True):
        kwargs_model_physical['source_light_model_list'].append('SERSIC')
        kwargs_model_physical['source_redshift_list'].append(redshifts['source2'])
        kwargs_model_physical['z_source'] = redshifts['source2']
        
    sim_g = SimAPI(numpix=numpix, kwargs_single_band=kwargs_g_band, kwargs_model=kwargs_model_physical)
    sim_r = SimAPI(numpix=numpix, kwargs_single_band=kwargs_r_band, kwargs_model=kwargs_model_physical)
    sim_i = SimAPI(numpix=numpix, kwargs_single_band=kwargs_i_band, kwargs_model=kwargs_model_physical)
    
    imSim_g = sim_g.image_model_class(kwargs_numerics)
    imSim_r = sim_r.image_model_class(kwargs_numerics)
    imSim_i = sim_i.image_model_class(kwargs_numerics)

    #lens mass model
    kwargs_mass = [{'sigma_v': lens['sigma'], 'center_x': 0, 'center_y': 0, 
                    'e1': lens['e1'], 'e2': lens['e2']}]
    kwargs_lens = sim_g.physical2lensing_conversion(kwargs_mass=kwargs_mass)
    
    #caustics and critical curves
    ra_crit_list, dec_crit_list, ra_caustic_list, dec_caustic_list = model['model_ext'].critical_curve_caustics(kwargs_lens, 
                                                                compute_window=15, grid_scale=0.01, center_x=0, center_y=0)
    if(len(ra_caustic_list)==0): return('No caustics')
    ra_caustic_list, dec_caustic_list = ra_caustic_list[0], dec_caustic_list[0]
    
    #cut on the Einstein radius of the lens
    if(kwargs_lens[0]['theta_E'] < cuts['E_min'] or kwargs_lens[0]['theta_E'] > cuts['E_max']):
        return ('No cut E')
    
    #First source light distributions & colors of the other bands for each source
    mag1, rad1, ellip1 = distribution(redshifts['source1'])
    n1 = random.uniform(0.3, 4.)
    
    #Pick center of first source on the caustic
    c1x, c1y = pick_center(ra_caustic_list, dec_caustic_list)
    theta_ra, theta_dec = model['solver'].image_position_from_source(c1x, c1y, kwargs_lens)
    magni1 = model['lensModel'].magnification(theta_ra, theta_dec, kwargs_lens)
    
    if(len(magni1)==0): return('No magni')
    
    #First source light:
    kwargs_source_mag_g_1 = [{'magnitude': mag1['mg'], 'R_sersic': rad1['rg'], 'n_sersic': n1,
                              'e1': ellip1['e1'], 'e2': ellip1['e2'], 'center_x': c1x, 'center_y': c1y}]
    #Adding color distribution to the bands (first source):
    kwargs_source_mag_r_1 = copy.deepcopy(kwargs_source_mag_g_1)
    kwargs_source_mag_r_1[0]['magnitude'], kwargs_source_mag_r_1[0]['R_sersic'] = mag1['mr'], rad1['rr']
    kwargs_source_mag_i_1 = copy.deepcopy(kwargs_source_mag_g_1)
    kwargs_source_mag_i_1[0]['magnitude'], kwargs_source_mag_i_1[0]['R_sersic'] = mag1['mi'], rad1['ri']
    #Same for second source
    if(double == True):
        mag2, rad2, ellip2 = distribution(redshifts['source2'])
        n2 = random.uniform(0.3, 4.)
        c2x, c2y = pick_center(ra_caustic_list, dec_caustic_list)
        theta_ra, theta_dec = model['solver'].image_position_from_source(c2x, c2y, kwargs_lens)
        magni2 = model['lensModel'].magnification(theta_ra, theta_dec, kwargs_lens)
        kwargs_source_mag_g_2 = [{'magnitude': mag2['mg'], 'R_sersic': rad2['rg'], 'n_sersic': n2, 
                              'center_x': c2x, 'center_y': c2y}]
        kwargs_source_mag_r_2 = copy.deepcopy(kwargs_source_mag_g_2)
        kwargs_source_mag_r_2[0]['magnitude'], kwargs_source_mag_r_2[0]['R_sersic'] = mag2['mr'], rad2['rr']
        kwargs_source_mag_i_2 = copy.deepcopy(kwargs_source_mag_g_2)
        kwargs_source_mag_i_2[0]['magnitude'], kwargs_source_mag_i_2[0]['R_sersic'] = mag2['mi'], rad2['ri']
        
    kwargs_source_mag_g = kwargs_source_mag_g_1 + kwargs_source_mag_g_2 if(double) else kwargs_source_mag_g_1 
    kwargs_source_mag_r = kwargs_source_mag_r_1 + kwargs_source_mag_r_2 if(double) else kwargs_source_mag_r_1 
    kwargs_source_mag_i = kwargs_source_mag_i_1 + kwargs_source_mag_i_2 if(double) else kwargs_source_mag_i_1 
    
    kwargs_lens_light_g, kwargs_source_g , point = sim_g.magnitude2amplitude(kwargs_lens_light_mag=None, 
                                                    kwargs_source_mag=kwargs_source_mag_g, kwargs_ps_mag=None)
    kwargs_lens_light_r, kwargs_source_r , point = sim_r.magnitude2amplitude(kwargs_lens_light_mag=None, 
                                                    kwargs_source_mag=kwargs_source_mag_r, kwargs_ps_mag=None)
    kwargs_lens_light_i, kwargs_source_i , point = sim_i.magnitude2amplitude(kwargs_lens_light_mag=None, 
                                                    kwargs_source_mag=kwargs_source_mag_i, kwargs_ps_mag=None)
    
    image_g = imSim_g.image(kwargs_lens, kwargs_source_g)
    image_r = imSim_r.image(kwargs_lens, kwargs_source_r)
    image_i = imSim_i.image(kwargs_lens, kwargs_source_i)
    
    image_g += sim_g.noise_for_model(model=image_g)
    image_r += sim_r.noise_for_model(model=image_r)
    image_i += sim_i.noise_for_model(model=image_i)
    
    image_g = image_g[::-1]
    image_r = image_r[::-1]
    image_i = image_i[::-1]
    
    if(np.isnan(np.sum(image_g))): 
        print('IS NAN!!!!!')
        return('IS NAN!!!!!')
    
    object_sim = np.array([image_g, image_r, image_i])
    if(double == False):
        m = calculate_magnitude(object_sim, 0) #band 0 = g band
        if(m > cuts['mmax'] or m < cuts['mmin']):
            return ('No cut mag')
    else:
        m = mag1['mg']
    
    b = (c1x**2 + c1y**2)/kwargs_lens[0]['theta_E']**2
    if(b<cuts['bmin'] or b > cuts['bmax']): return('No cut b')
    
    parameters = {'zl/z1': [redshifts['lens']/redshifts['source1']], 'm': [m], 'b': [b], 'E': [kwargs_lens[0]['theta_E']], 'Magni 1': [max(np.abs(magni1))]}
    
    write_files(coadd_id, image_g, image_r, image_i, parameters)  
    
    """
    rgb = make_lupton_rgb(np.log10(image_g), np.log10(image_r), np.log10(image_i), Q=2., stretch=4.)
    plt.figure()
    plt.imshow(rgb)
    plt.xticks([], [])
    plt.yticks([], [])
    #plt.savefig('Image' + str(i) + '.png', bbox_inches='tight')
    plt.show(block=True)
    #plt.close()
    """
    
    return ('ok')
    

In [7]:
complete_data = pd.read_csv('Data/all_data_sim.csv')
print(len(complete_data))

20265


In [8]:
def simulations_from_data(num_max, double):
    num_sim = 0
    complete_data = pd.read_csv('Data/all_data_sim.csv')
    data = complete_data.sample(n = 40000, replace = True).reset_index() #(frac=1)
    
    lens_model_list = ['SIE']
    lensModel = LensModel(lens_model_list=lens_model_list)
    lensModelExt = LensModelExtensions(lensModel)
    solver = LensEquationSolver(lensModel)
    
    model = {'lensModel': lensModel, 'model_ext': lensModelExt, 'solver': solver}
    
    for index, row in data.iterrows():
        if index == num_max:
            break
        DES_g_band_obs['seeing'] = row['FWHM_WMEAN_G'] 
        DES_r_band_obs['seeing'] = row['FWHM_WMEAN_R']
        DES_i_band_obs['seeing'] = row['FWHM_WMEAN_I']
        
        coadd_id = int(row['COADD_OBJECT_ID'])
        z_lens = row['Z']
        z1_max = random.uniform(0.4, 0.8)
        z_source1 = random.uniform(z_lens, z_lens + z1_max) # 1.2
        z_source2 = random.uniform(z_source1, 1.7) # 0.35
        redshifts = {'lens': z_lens, 'source1': z_source1, 'source2': z_source2}
        
        angle, ratio = param.ellipticity2phi_q(row['SOF_CM_G_1'], row['SOF_CM_G_2'])
        angle += 0.698132*random.uniform(-1, 1) #noise between -40 and 40 degrees
        ratio = random.uniform(0.001, 1) #distribution for the axis ratio
        e1, e2 = param.phi_q2_ellipticity(angle, ratio)
        lens = {'sigma': row['VEL_DISP'] + 150, 'e1': e1, 'e2': e2}
    
        mmin, mmax, E_min, E_max, b_min, b_max = 15, 24, 0.4, 4., 0, 10
        cuts = {'E_max': E_max, 'E_min': E_min, 'mmax': mmax, 'mmin': mmin, 'bmin': b_min, 'bmax': b_max}
        
        status = simulation(model, coadd_id, redshifts, lens, cuts, double)
        #print(status)
        while(status == 'No cut mmag'):
            z1_max = random.uniform(0.4, 0.8)
            z_source1 = random.uniform(z_lens, z_lens + z1_max)
            redshifts = {'lens': z_lens, 'source1': z_source1, 'source2': z_source2}
            status = simulation(coadd_id, redshifts, lens, cuts, double)
            #print(status)
        if(status == 'ok'):
            num_sim += 1
    print(num_sim)
            

In [9]:
bell -n notify simulations_from_data(num_max = 2500, double=False)

KeyboardInterrupt: 

AttributeError: 'KeyboardInterrupt' object has no attribute 'message'

In [ ]:
bell -n say

In [ ]:
#! convert -delay 50 -loop 0 *.png Only_sources_data.gif
#! rm *.png